In [ ]:
from ner.Datasets.utils import *
from ner.Datasets.Conll2003Dataset import *

%load_ext autoreload
%autoreload 2

In [ ]:
# data_tr = Conll2003Dataset(utils, split = 'train')
# data_te = Conll2003Dataset(utils, split = 'test')

In [ ]:
clean_data_te = Conll2003Dataset(split = 'test', cleaned = True)
clean_data_tr = Conll2003Dataset(split = 'train', cleaned = True)

In [ ]:
clean_data_te

In [ ]:
save_conll_dataset(clean_data_te)
save_conll_dataset(clean_data_tr)

In [ ]:
dataset_test = load_conll_dataset(cleaned= True)

In [ ]:
dataset_train.dataset

## Preparing datasets for finetuning

In [ ]:
from ner.llm_ner.prompt_techniques import *

runs = 2000
pt = PT_GPT_NER(None)
res2 = pt.process_dataset_for_finetuning(runs=runs)
pt = PT_OutputList(None)
res2 = pt.process_dataset_for_finetuning(runs=runs)
pt = PT_Wrapper(None)
res2 = pt.process_dataset_for_finetuning(runs=runs)

## Testing few-shots

In [ ]:
# dataset_train, dataset_llama_50 = load_conll_dataset(split = 'test')
# dataset_test = load_conll_dataset(split = 'test', cleaned=True)
dataset_test_train, dataset_test_test = dataset_test.train_test_split(test_size=20)

In [ ]:
from ner.Llama2_NER import Llama2_NER_FewShots_Entity
ner_ent = Llama2_NER_FewShots_Entity(None, utils, dataset_test_train, nb_few_shots= 16, with_verify=False, prompt_type="discussion")

In [ ]:
# dataset_llama_50.select([8,10,22,24,25,44,49])
for i, sample in enumerate(dataset_test_test) :
    print(sample['text'])
    print(sample['spans'])
    print(ner_ent.get_prompts(sample['text'], [], 5))
    print()
    print("----------------------------------------------------")
    print()